# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

In [28]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pprint import pprint
from datetime import datetime, timedelta

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [2]:
# определяем список ключевых слов, которые нам интересны
KEYWORDS = ['python', 'парсинг', 'запрос', 'фреймворк', 'комьюнити', 'видео', 'android', 'linux']

In [3]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
# извлекаем посты
posts = soup.find_all('article', class_='post')

In [34]:
def get_dates(date):
#     функция которая прописывает цифровой вид даты
    date_format = '%Y-%d-%m'
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    if 'сегодня' in date: 
        return date.replace('сегодня в', today.strftime(date_format))
    else:
        return date.replace('вчера в', yesterday.strftime(date_format))

In [35]:
habr_news = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    if post.find(class_='post__text'): 
        article = post.find(class_='post__text').text
    else:
        continue
    article_lower = article.lower()
   # ищем вхождение хотя бы одного желаемого слова
    for keyword in KEYWORDS:
        if keyword in article_lower:
            time_element = post.find(class_='post__time')
            title_element = post.find('a', class_='post__title_link')
            row = {'date': pd.to_datetime(get_dates(time_element.text)), 'title': title_element.text, 'href': title_element.attrs.get('href'), "keyword": keyword}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

,date,title,href,keyword
0,2020-05-10 09:51:00,За двумя мобильными сервисами: HMS и GMS в одн...,https://habr.com/ru/company/cardsmobile/blog/5...,android
0,2020-04-10 21:51:00,Intel наносит ответный удар,https://habr.com/ru/post/521998/,видео


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [36]:
habr_news_full = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
#     из поста мы можем вытащить линк на статью и провалиться по ней
    title_element = post.find('a', class_='post__title_link')
    link = title_element.attrs.get('href')
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
#     обнаружились разные классы в теге и отмена публикаций
    if soup.find(class_='post__text'): 
        articles_text = soup.find(class_='post__text').text
    else:
        continue
#     ищем ключевые слова в теле статьи
    for keyword in KEYWORDS:
        if keyword in articles_text.lower():
            time_element = post.find(class_='post__time')
            title_element = post.find('a', class_='post__title_link')
            row = {'date':  pd.to_datetime(get_dates(time_element.text)), 'title': title_element.text, 'href': title_element.attrs.get('href'), 'article': articles_text, "keyword": keyword}
            habr_news_full = pd.concat([habr_news_full, pd.DataFrame([row])]) 
            break
habr_news_full


,date,title,href,article,keyword
0,2020-05-10 10:46:00,Как Magento 2 взаимодействует с Vue Storefront,https://habr.com/ru/company/rshb/blog/521692/,\nПривет! Меня зовут Павел и я занимаюсь бэкен...,запрос
0,2020-05-10 09:51:00,За двумя мобильными сервисами: HMS и GMS в одн...,https://habr.com/ru/company/cardsmobile/blog/5...,"\n\r\nПривет, Хабр! Меня зовут Андрей, я делаю...",android
0,2020-05-10 09:44:00,Приёмы и хитрости начинающего Java-программиста,https://habr.com/ru/post/520306/,"Небольшая коллекция практик, трюков и подсказо...",linux
0,2020-05-10 09:14:00,Как это устроено: робот Spot от Boston Dynamics,https://habr.com/ru/post/521878/,"Spot танцуют, стойко сносят пинки, буксируют г...",python
0,2020-05-10 09:14:00,Лицом к разработчикам: модернизировать частное...,https://habr.com/ru/company/jetinfosystems/blo...,Сложно ли создать виртуальную машину (ВМ) в об...,запрос
0,2020-05-10 08:26:00,"2. UserGate Getting Started. Требования, устан...",https://habr.com/ru/company/tssolution/blog/52...,"Здравствуйте, это вторая статья о NGFW решении...",запрос
0,2020-04-10 23:29:00,Дайджест свежих материалов из мира фронтенда з...,https://habr.com/ru/post/522004/,Предлагаем вашему вниманию подборку с ссылками...,запрос
0,2020-04-10 22:26:00,Систематизированный список всех функций Java и...,https://habr.com/ru/post/521988/,"Последнее обновление: 29.09.2020, вклющее изме...",фреймворк
0,2020-04-10 21:51:00,Intel наносит ответный удар,https://habr.com/ru/post/521998/,\n\nПосле более чем десятилетнего превосходств...,видео


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [7]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
def get_leaked_data(EMAIL):
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    data = {
        'emailAddresses': EMAIL
    }
    data = json.dumps(data)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 YaBrowser/20.9.0.928 Yowser/2.5 Safari/537.36',
                'Vaar-Header-App-Product': 'hackcheck-web-avast',
                'Vaar-Version': '0'}
    req = requests.post(URL, data=data, verify=False,  headers=headers)
    json_ = json.loads(req.text)
    return json_

# pprint(json.loads(req.text))


In [8]:
breaches = get_leaked_data(EMAIL)['breaches']
data = get_leaked_data(EMAIL)['data']
df_breaches = pd.DataFrame()
df_data = pd.DataFrame()
for key, value in breaches.items():
    row_breaches = {'description': value['description'], 'site': value['site'], 'breach_id': int(key),
                    'date': pd.to_datetime(value['publishDate'])}
    df_breaches = pd.concat([df_breaches, pd.DataFrame([row_breaches])])
for key, value in data.items():
    for k, v in value.items():
        row_data = {'e-mail': k, 'site': key, 'breach_id': v[0]['breachId']}
        df_data = pd.concat([df_data, pd.DataFrame([row_data])])

df_leaked = df_data.merge(df_breaches, on=['breach_id', 'site'], how='left')
df_leaked.head()


C:\Users\fines\anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\fines\anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,e-mail,site,breach_id,description,date
0,xxx@x.ru,parapa.mail.ru,3176,"In July and August 2016, two criminals execute...",2017-02-14 00:00:00+00:00
1,xxx@x.ru,verifications.io,16613,Big data e-mail verification platform verifica...,2019-03-28 00:00:00+00:00
2,yyy@y.com,verifications.io,16613,Big data e-mail verification platform verifica...,2019-03-28 00:00:00+00:00
3,xxx@x.ru,adobe.com,3,"In October of 2013, criminals penetrated Adobe...",2016-10-21 00:00:00+00:00
4,yyy@y.com,adobe.com,3,"In October of 2013, criminals penetrated Adobe...",2016-10-21 00:00:00+00:00


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [9]:
from yaml import load
f = open('config_vk.yaml', mode = 'r', encoding = 'utf-8') 
config = load(f)
token = config['access_token']

C:\Users\fines\anaconda3\lib\site-packages\ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
url = 'https://api.vk.com/method/wall.get'
params = {'domain': 'netology',
          'v': 5.103,
          'count': 200,
          'access_token': token}

In [11]:
r = requests.get(url, params = params)
data = r.json()
pprint(data['response']['items'])

[{'attachments': [{'photo': {'access_key': 'e7fdb64be3beea8758',
                             'album_id': -7,
                             'date': 1601027163,
                             'has_tags': False,
                             'id': 457245349,
                             'owner_id': -30159897,
                             'post_id': 52999,
                             'sizes': [{'height': 87,
                                        'type': 'm',
                                        'url': 'https://sun9-24.userapi.com/57XKYnz9zeE3Qem0XkS25osRcKeHnmAOxf7OYQ/1jTXGMVyJlY.jpg',
                                        'width': 130},
                                       {'height': 87,
                                        'type': 'o',
                                        'url': 'https://sun9-24.userapi.com/57XKYnz9zeE3Qem0XkS25osRcKeHnmAOxf7OYQ/1jTXGMVyJlY.jpg',
                                        'width': 130},
                                       {'height': 133,
   

                             'sizes': [{'height': 130,
                                        'type': 'm',
                                        'url': 'https://sun9-71.userapi.com/T1tYjgp_cUK941ecpDkyk4_XQJkA9etDG2jDBA/zXpFrj2ZrbE.jpg',
                                        'width': 130},
                                       {'height': 130,
                                        'type': 'o',
                                        'url': 'https://sun9-71.userapi.com/T1tYjgp_cUK941ecpDkyk4_XQJkA9etDG2jDBA/zXpFrj2ZrbE.jpg',
                                        'width': 130},
                                       {'height': 200,
                                        'type': 'p',
                                        'url': 'https://sun9-51.userapi.com/KH4BbKIYymUsrghiso7cZWLRVfpewZT5sxd7nw/_EILSh0Rej0.jpg',
                                        'width': 200},
                                       {'height': 320,
                                        'type': 'q',
    

                            'title': 'Дополненная реальность (AR): что это и '
                                     'какую пользу приносит бизнесу | Медиа '
                                     'Нетологии: образователь',
                            'url': 'http://netolo.gy/fSx'},
                   'type': 'link'}],
  'comments': {'can_post': 1, 'count': 21, 'groups_can_post': True},
  'date': 1601539680,
  'from_id': -30159897,
  'id': 53243,
  'likes': {'can_like': 1, 'can_publish': 1, 'count': 12, 'user_likes': 0},
  'marked_as_ads': 0,
  'owner_id': -30159897,
  'post_source': {'type': 'vk'},
  'post_type': 'post',
  'reposts': {'count': 1, 'user_reposted': 0},
  'text': 'Ещё несколько лет назад все ловили покемонов на свои смартфоны, а '
          'сегодня эта же технология приносит реальную пользу бизнесу 🤯 \n'
          ' \n'
          'Разбираемся в блоге, что такое дополненная реальность и как её '
          'применяют в маркетинге → http://netolo.gy/fSx\n'
          '\n'
    

                                                 'type': 'a',
                                                 'url': 'https://sun9-50.userapi.com/qxIwaGkhXqHEnEpTF8cZvmMfPx-ok8k_yMuUyw/7Q8FAa6KGXM.jpg',
                                                 'width': 200},
                                                {'height': 179,
                                                 'type': 'b',
                                                 'url': 'https://sun9-55.userapi.com/M0Sa5RI2U_iSFPco7GI1T7-PjMAChUopTUIEhw/17Z0E8KNRQs.jpg',
                                                 'width': 400},
                                                {'height': 200,
                                                 'type': 'c',
                                                 'url': 'https://sun9-54.userapi.com/j6dOTPOV7VqfadW0wggHE8n5yQaCnuP90JBJyQ/7fhHB9_rcmk.jpg',
                                                 'width': 200},
                                                {'height': 100,
    

                                        'url': 'https://sun9-5.userapi.com/5gb380a5y1D0b81LmaIL-BRRkxFyVOvYhonsOA/AFil3WrB2Po.jpg',
                                        'width': 130},
                                       {'height': 200,
                                        'type': 'p',
                                        'url': 'https://sun9-35.userapi.com/qybT1rXleZxJ6D2ippeEPC5MPwGMYxErouFzbw/oBh-Tngw0qk.jpg',
                                        'width': 200},
                                       {'height': 320,
                                        'type': 'q',
                                        'url': 'https://sun9-40.userapi.com/CopbSbsk3YV4bFYV1QJhnOBs02k7YXNik0WVjg/w0uUaORShFM.jpg',
                                        'width': 320},
                                       {'height': 510,
                                        'type': 'r',
                                        'url': 'https://sun9-24.userapi.com/BuH9lbcxV9AwmnzgiNNrR8jm0t2UNIY5Cqcko

...

                             'sizes': [{'height': 130,
                                        'type': 'm',
                                        'url': 'https://sun9-21.userapi.com/DKQQCdcQzwSooDOmEGv2kC2982dAcH6xZrc5VA/tkLX-5vCZRE.jpg',
                                        'width': 130},
                                       {'height': 130,
                                        'type': 'o',
                                        'url': 'https://sun9-21.userapi.com/DKQQCdcQzwSooDOmEGv2kC2982dAcH6xZrc5VA/tkLX-5vCZRE.jpg',
                                        'width': 130},
                                       {'height': 200,
                                        'type': 'p',
                                        'url': 'https://sun9-22.userapi.com/tdYXAnqgGq_FN05b3pG29vAyf9glkbaHf7OFUQ/UZ6Vq3-uPpk.jpg',
                                        'width': 200},
                                       {'height': 320,
                                        'type': 'q',
    

                                                {'height': 240,
                                                 'type': 'o',
                                                 'url': 'https://sun9-6.userapi.com/OA792jLJ2fj76wLmyAydMkzI3k9_Hcze2uYA1Q/Ll46fDCIfi4.jpg',
                                                 'width': 537}],
                                      'text': '',
                                      'user_id': 100},
                            'title': 'Как технологии меняют образование | '
                                     'Медиа Нетологии: университет '
                                     'интернет-профессий',
                            'url': 'http://netolo.gy/fLR'},
                   'type': 'link'}],
  'comments': {'can_post': 1, 'count': 0, 'groups_can_post': True},
  'date': 1598282460,
  'from_id': -30159897,
  'id': 50483,
  'likes': {'can_like': 1, 'can_publish': 1, 'count': 21, 'user_likes': 0},
  'marked_as_ads': 0,
  'owner_id': -30159897,
  'post_so

                            'photo': {'album_id': -66,
                                      'date': 1597999334,
                                      'has_tags': False,
                                      'id': 457245130,
                                      'owner_id': -30159897,
                                      'sizes': [{'height': 73,
                                                 'type': 'm',
                                                 'url': 'https://sun9-24.userapi.com/pUCoZQA8Wcf653YpUclzV-kjIzCC9SX11iX_Ug/NOyrz7tZuHY.jpg',
                                                 'width': 130},
                                                {'height': 87,
                                                 'type': 'o',
                                                 'url': 'https://sun9-46.userapi.com/q1eqI_vK07k1fMjXOCm58pdKvPKAGmxnRI1Xzw/0we6-2yYw4E.jpg',
                                                 'width': 130},
                                                {'he

                                      'sizes': [{'height': 89,
                                                 'type': 'a',
                                                 'url': 'https://sun9-75.userapi.com/XDrSwRzs_H5lDRQRajNEBOLXWHTPJHHg5tZ5KQ/VzgsUD3vkmU.jpg',
                                                 'width': 200},
                                                {'height': 179,
                                                 'type': 'b',
                                                 'url': 'https://sun9-67.userapi.com/G3GJmcI8ts-9n2YP82sCbcPFvCgwulL-FsAcZg/QRtObKtYH6w.jpg',
                                                 'width': 400},
                                                {'height': 200,
                                                 'type': 'c',
                                                 'url': 'https://sun9-50.userapi.com/ac5_sw7lyz24AJEmuqxJLqjWziOM3GMSWb7W_Q/-knCwt7KVTA.jpg',
                                                 'width': 200},
     

                                        'width': 1080}],
                             'text': '',
                             'user_id': 100},
                   'type': 'photo'},
                  {'photo': {'access_key': '668fc040da9d75e77a',
                             'album_id': -7,
                             'date': 1597139563,
                             'has_tags': False,
                             'id': 457244490,
                             'owner_id': -30159897,
                             'sizes': [{'height': 130,
                                        'type': 'm',
                                        'url': 'https://sun9-65.userapi.com/yTiIgeKlFhSy_iByepDQTbIsBKWLaFhCfDDAQQ/Ff9xymbqt7o.jpg',
                                        'width': 130},
                                       {'height': 130,
                                        'type': 'o',
                                        'url': 'https://sun9-65.userapi.com/yTiIgeKlFhSy_iByepDQTbIsBKWLaFhCfD

In [12]:
df_vk = pd.DataFrame()
for i, post in enumerate (data['response']['items']):
    row_data = {'text': data['response']['items'][i]['text'], 
                'date': pd.to_datetime(data['response']['items'][i]['date'], unit = 's')}
    df_vk = pd.concat([df_vk, pd.DataFrame([row_data])])
df_vk

,text,date
0,Всё ещё дарим подарки 🎁\n \nКаждую неделю разы...,2020-09-28 08:00:10
0,20 октября стартует бесплатный марафон «Первая...,2020-10-05 07:08:00
0,Собрали для вас подборку выступлений с конфере...,2020-10-04 13:54:00
0,*партнёрский пост* \n \nИнтернет-издание «Лайк...,2020-10-04 07:24:00
0,В новой афише интересных онлайн-мероприятий: ч...,2020-10-03 12:54:00
...,...,...
0,"Друзья, последнее пятое задание уже вышло, и с...",2020-08-19 10:00:06
0,Почувствовать себя дизайнером стало совсем про...,2020-08-19 07:50:00
0,"Исследования показывают, что люди проводят поч...",2020-08-18 14:42:00
0,Осталась всего одна ступень до вершины!⚡\n \nВ...,2020-08-18 09:00:11


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
